In [24]:
# from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import numpy as np
import math

In [25]:
import data_file
male_list = data_file.male_list
female_list = data_file.female_list

In [26]:
from transformers import BertTokenizer, BertModel
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
# # gendered_word_list
# gendered_word_list = [["she", "he"],["female", "male"], ["woman", "man"], ["girl", "boy"], ["sister", "brother"], ["daughter", "son"], ["mother", "father"]]
XX = ["female", "woman", "girl", "sister", "daughter", "mother", "aunt", "grandmother", "daughter"]
YY = ["male", "man", "boy", "brother", "son", "father", "uncle", "grandfather", "son"]

In [28]:
templates = ["this is the _.", "that is the _.", "there is the _.", "here is the _.", "the _ is here.", "the _ is there.", "the _ is a person.", "it is the _.","the _ is a [MASK].", "the _ is an engineer.", "the _ is a nurse with superior technical skills."]

In [29]:
temps = ["temp1", "temp2", "temp3", "temp4","temp5", "temp6", "temp7", "temp8", "temp9", "temp10", "temp11"]

In [30]:
len(templates)

11

In [31]:
def get_index(sentence, word):
    toks = tokenizer(sentence).input_ids
    wordpieces = tokenizer(word).input_ids
    word = wordpieces[1] # use first wordpiece
    for i, t in enumerate(toks):
        if t == word:
            return i

In [32]:
# Vulic
def sentence_embedding(template, word):
    template = "_"
    sentence = template.replace("_", word)
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs, output_hidden_states=True)
    hidden_states_1_4 = outputs["hidden_states"][1:5]
    hidden_states = torch.stack([torch.FloatTensor(i[0]) for i in hidden_states_1_4])
    mean_embeddings = torch.mean(hidden_states[:,1:-1,:],0)
    mean_embeddings = torch.mean(mean_embeddings,0)
    #         print((mean_embeddings.shape))
    return mean_embeddings.cpu().detach().numpy()


In [33]:
def cossim(x, y):
    return np.dot(x, y) / math.sqrt(np.dot(x, x) * np.dot(y, y))

In [34]:
def construct_cossim_lookup(XY, AB):
    """
    XY: mapping from target string to target vector (either in X or Y)
    AB: mapping from attribute string to attribute vectore (either in A or B)
    Returns an array of size (len(XY), len(AB)) containing cosine similarities
    between items in XY and items in AB.
    """

    cossims = np.zeros((len(XY), len(AB)))
    for xy in XY:
        for ab in AB:
#             print(XY[xy].shape)
#             print(AB[ab].shape)
            cossims[xy, ab] = cossim(XY[xy], AB[ab])
    return cossims


In [35]:
def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [36]:

def s_XAB(A, s_wAB_memo):
    return s_wAB_memo[A].sum()

In [37]:
def s_wAB(X, Y, cossims):
    """
    Return vector of s(w, A, B) across w, where
        s(w, A, B) = mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
    """
#     print((cossims[X, :].mean(axis=0) - cossims[Y, :].mean(axis=0)).shape)
    return cossims[X, :].mean(axis=0) - cossims[Y, :].mean(axis=0)

In [38]:
# def s_wAB_df(X, Y, cossims):
#     """
#     Return vector of s(w, A, B) across w, where
#         s(w, A, B) = mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
#     """
# #     print(cossims[:, A].mean(axis=1))
#     df1 = pd.DataFrame(cossims[X, :].mean(axis=0))
#     df2 = pd.DataFrame(cossims[Y, :].mean(axis=0))
#     return df1, df2


def s_XAB_df(A, B, s_wAB_memo):
    df1 = pd.DataFrame(s_wAB_memo[A])
    df2 = pd.DataFrame(s_wAB_memo[B])
    return df1, df2

In [39]:
def s_XYAB(A, B, s_wAB_memo):
    r"""
    Given indices of target concept X and precomputed s_wAB values,
    the WEAT test statistic for p-value computation.
    """
    return s_XAB(A, s_wAB_memo) - s_XAB(B, s_wAB_memo)

In [40]:
# def WEAT_test(X, Y, A, B, n_samples, cossims, parametric=False):
def WEAT_test(X, Y, A, B, n_samples, cossims):
    ''' Compute the p-val for the permutation test, which is defined as
        the probability that a random even partition X_i, Y_i of X u Y
        satisfies P[s(X_i, Y_i, A, B) > s(X, Y, A, B)]
    '''
    X = np.array(list(X), dtype=np.int)
    Y = np.array(list(Y), dtype=np.int)
    A = np.array(list(A), dtype=np.int)
    B = np.array(list(B), dtype=np.int)

    assert len(X) == len(Y)
    size = len(X)
    s_wAB_memo = s_wAB(X, Y, cossims=cossims)
#     print(s_wAB_memo)
    XY = np.concatenate((X, Y))

#     if parametric:
#     log.info('Using parametric test')
    s = s_XYAB(A, B, s_wAB_memo)
    return s


def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [41]:
def get_effect_size(df1, df2, k=0):
    diff = (df1[k].mean() - df2[k].mean())
    std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
    return diff / std_

In [42]:
template_score_dict = {}
for ind, template in enumerate(templates):
    score_dict = {}
    attribute_template = template
    target_template = template
#     for model_ind in range(len(models)):
#     for model_ind in [2]:
    X = {"x" + str(j): sentence_embedding(attribute_template, j) for j in XX}
    Y = {"y" + str(j): sentence_embedding(attribute_template, j) for j in YY}
    (X, Y) = convert_keys_to_ints(X, Y)
    XY = X.copy()
    XY.update(Y)
    X = np.array(list(X), dtype=np.int)
    Y = np.array(list(Y), dtype=np.int)
    for i in range(len(female_list)):
        AA = female_list[i]
    #     print(AA)
    #     print(XX)
        BB = male_list[i]

        A = {"a" + str(j): sentence_embedding(target_template, j) for j in AA}
        B = {"b" + str(j): sentence_embedding(target_template, j) for j in BB}

        (A, B) = convert_keys_to_ints(A, B)


        AB = A.copy()
        AB.update(B)

        cossims = construct_cossim_lookup(XY, AB)
        A = np.array(list(A), dtype=np.int)
        B = np.array(list(B), dtype=np.int)


        s_wAB_memo = s_wAB(X, Y, cossims=cossims)
        df1,df2 = s_XAB_df(A, B, s_wAB_memo)
        effect_size = get_effect_size(df1, df2)
        score_dict[i] = effect_size
        print(score_dict[i])
    template_score_dict[temps[ind]] = score_dict

0.871220599351518
1.0582848989647022
1.3139520609331834
0.8155196422979215
1.3221400872999771
0.9451984942539693
0.9105468943341706
1.438234522058761
0.9077463010465836
1.1736983779082273
0.8851260733511905
1.2028879810670807
0.990269243051717
1.1613301937896046
1.2099159497682055
0.755754650857251
0.7922075331177535
1.0890836980923013
0.8894199254110223
0.8198236233389656
0.871220599351518
1.0582848989647022
1.3139520609331834
0.8155196422979215
1.3221400872999771
0.9451984942539693
0.9105468943341706
1.438234522058761
0.9077463010465836
1.1736983779082273
0.8851260733511905
1.2028879810670807
0.990269243051717
1.1613301937896046
1.2099159497682055
0.755754650857251
0.7922075331177535
1.0890836980923013
0.8894199254110223
0.8198236233389656
0.871220599351518
1.0582848989647022
1.3139520609331834
0.8155196422979215
1.3221400872999771
0.9451984942539693
0.9105468943341706
1.438234522058761
0.9077463010465836
1.1736983779082273
0.8851260733511905
1.2028879810670807
0.990269243051717
1.16

In [43]:
import json
a_file = open("6_seat_vulic_templates_Language_models.json", "w")
json.dump(template_score_dict, a_file)
a_file.close()